In [12]:
import os
import shutil
import subprocess
from glob import glob

In [3]:
import rasterio

In [4]:
data = os.getcwd() + "\\data"

In [5]:
months_list = glob(data+"\\*")

In [6]:
months_list

['C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\dec_data',
 'C:\\Users\\Hello\\Documents\\remote-sensing\\APIs\\SentinelSat\\data\\jan_data']

In [8]:
for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    num_scenes = len(data_list)
    i = 0
    for scene in data_list:
        path  =  scene + "\\GRANULE"
        data_folder = glob(path + "\\*")
        data_folder[0] =  data_folder[0] + "\\IMG_DATA\\R10m"
        files = glob(data_folder[0] + "\\*.jp2")
        i = i + 1
        for file in files:
            dst = file.split('\\')[-1].split('.')[0] + ".tif"
            dst = data_folder[0] +"\\"+ dst
            if not os.path.exists(dst):
                os.system("gdal_translate "+file+" "+dst)
        if os.path.exists(dst):
            print("Converted (" + str(i) + "/" + str(num_scenes) + ") " + scene.split('\\')[-1].split('.')[0] + "...")
        else:
            print("Failed Converting (" + str(i) + "/" + str(num_scenes) + ") " + scene.split('\\')[-1].split('.')[0] + "...")

C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\dec_data
Converted (1/23) S2A_MSIL2A_20181215T051211_N0211_R019_T44PLC_20181215T073500...
Converted (2/23) S2A_MSIL2A_20181215T051211_N0211_R019_T44PMC_20181215T073500...
Converted (3/23) S2A_MSIL2A_20181222T050211_N0211_R119_T44PLC_20181222T072528...
Converted (4/23) S2A_MSIL2A_20181222T050211_N0211_R119_T44PMC_20181222T072528...
Converted (5/23) S2A_MSIL2A_20181222T050211_N0211_R119_T44PNC_20181222T072528...
Converted (6/23) S2A_MSIL2A_20181222T050211_N0211_R119_T44QMD_20181222T072528...
Converted (7/23) S2A_MSIL2A_20181222T050211_N0211_R119_T44QND_20181222T072528...
Converted (8/23) S2A_MSIL2A_20181225T051221_N0211_R019_T44PLC_20181225T073544...
Converted (9/23) S2A_MSIL2A_20181225T051221_N0211_R019_T44PMC_20181225T073544...
Converted (10/23) S2B_MSIL2A_20181217T050219_N0211_R119_T44PLC_20181217T084211...
Converted (11/23) S2B_MSIL2A_20181217T050219_N0211_R119_T44PMC_20181217T084211...
Converted (12/23) S2B_MSIL2A_2018121

In [11]:
if not os.path.exists(os.getcwd()+"\\stacked_images"):
    stack_folder = os.makedirs(os.getcwd()+"\\stacked_images")
for month in months_list:
    dst = month.split('\\')[-1]
    if not os.path.exists(os.getcwd()+"\\stacked_images\\"+dst):
        os.makedirs(os.getcwd()+"\\stacked_images\\"+dst)

In [13]:
for month in months_list:
    print(month)
    data_list = glob(month+"\\*")
    num_scenes = len(data_list)
    i = 0
    for scene in data_list:
        i = i + 1
        path  =  scene + "\\GRANULE"
        data_folder = glob(path + "\\*")
        data_folder[0] =  data_folder[0] + "\\IMG_DATA\\R10m"
        tifFiles = []
        tifFiles.append(glob(data_folder[0] + "\\*B08_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B04_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B03_10m.tif")[0])
        tifFiles.append(glob(data_folder[0] + "\\*B02_10m.tif")[0])
        with rasterio.open(tifFiles[0]) as src0:
            meta = src0.meta
        meta.update(count = len(tifFiles))
        dest = tifFiles[0].split('\\')[-1].split('.')[0].split('_')[:2]
        dest_name = dest[0] + "_" + dest[1] + ".tif"
        dest = data_folder[0] +"\\"+ dest_name
        if not os.path.exists(dest):
            with rasterio.open(dest, 'w', **meta) as dst:
                for id, layer in enumerate(tifFiles, start=1):
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
        print("(" + str(i)  + "/" + str(num_scenes) + ") Stacked " + dest_name + "...")
        stack_dest_name = scene.split("\\")[-1].split('.')[0] + ".tif"
        stack_dest = os.getcwd() + "\\stacked_images\\"+ month.split('\\')[-1] + "\\" + stack_dest_name
        if not os.path.exists(stack_dest):
            shutil.copy2(dest, stack_dest)
        print("(" + str(i)  + "/" + str(num_scenes) + ") Copied to stacked_data ...")

C:\Users\Hello\Documents\remote-sensing\APIs\SentinelSat\data\dec_data
(1/23) Stacked T44PLC_20181215T051211.tif...
(1/23) Copied to stacked_data ...
(2/23) Stacked T44PMC_20181215T051211.tif...
(2/23) Copied to stacked_data ...
(3/23) Stacked T44PLC_20181222T050211.tif...
(3/23) Copied to stacked_data ...
(4/23) Stacked T44PMC_20181222T050211.tif...
(4/23) Copied to stacked_data ...
(5/23) Stacked T44PNC_20181222T050211.tif...
(5/23) Copied to stacked_data ...
(6/23) Stacked T44QMD_20181222T050211.tif...
(6/23) Copied to stacked_data ...
(7/23) Stacked T44QND_20181222T050211.tif...
(7/23) Copied to stacked_data ...
(8/23) Stacked T44PLC_20181225T051221.tif...
(8/23) Copied to stacked_data ...
(9/23) Stacked T44PMC_20181225T051221.tif...
(9/23) Copied to stacked_data ...
(10/23) Stacked T44PLC_20181217T050219.tif...
(10/23) Copied to stacked_data ...
(11/23) Stacked T44PMC_20181217T050219.tif...
(11/23) Copied to stacked_data ...
(12/23) Stacked T44PNC_20181217T050219.tif...
(12/23) Co